# Homework: Vector Search

In [39]:
from fastembed import TextEmbedding
import pandas as pd
from qdrant_client import QdrantClient, models

### Q1. Embedding the query
Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

You should get a numpy array of size 512.

What's the minimal value in this array?

In [7]:
text = 'I just discovered the course. Can I join now?'
model_handle = "jinaai/jina-embeddings-v2-small-en"
text_model = TextEmbedding(
        model_name=model_handle,
)
text_embeddings = list(text_model.embed(text))

In [10]:
len(text_embeddings[0])

512

In [12]:
text_embeddings[0].min()

np.float64(-0.11726373551188797)

### Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can chech that by using the norm function:

In [14]:
import numpy as np
np.linalg.norm(text_embeddings[0])

np.float64(1.0)

Which means that we can simply compute the dot product between two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

In [15]:
text_embeddings[0].dot(text_embeddings[0])

np.float64(1.0)

### Q2. Cosine similarity with another vector
What's the cosine similarity between the vector for the query and the vector for the document?

In [16]:
doc = 'Can I still join the course after the start date?'

In [17]:
query_embeddings = list(text_model.embed(doc))[0]

In [18]:
text_embeddings[0].dot(query_embeddings)

np.float64(0.9008528856818037)

### Q3. Ranking by cosine
Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

In [19]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [22]:
embeddings = []

for d in documents:
    embedding = list(text_model.embed(d['text']))[0]
    embeddings.append(embedding)
V = np.vstack(embeddings)

Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:

In [24]:
V.dot(query_embeddings)

array([0.8106057 , 0.84989067, 0.7991335 , 0.68130469, 0.6739288 ])

In [25]:
np.argmax(V.dot(query_embeddings))

np.int64(1)

### Q4. Ranking by cosine, version two
Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?\
Is it different from Q3? If yes, why?

In [27]:
embeddings = []

for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    embedding = list(text_model.embed(full_text))[0]
    embeddings.append(embedding)
V = np.vstack(embeddings)

In [28]:
V.dot(query_embeddings)

array([0.92732519, 0.85948473, 0.83966023, 0.76320296, 0.76044492])

In [29]:
np.argmax(V.dot(query_embeddings))

np.int64(0)

Different from Q3 because question exact match question from text 0

### Q5. Selecting the embedding model
Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

In [36]:
supported_models = (
    pd.DataFrame(TextEmbedding.list_supported_models())
    .sort_values("size_in_GB")
    .drop(columns=["sources", "model_file", "additional_files"])
    .reset_index(drop=True)
)
supported_models.head()

,model,description,license,size_in_GB,dim,tasks
0,BAAI/bge-small-en-v1.5,"Text embeddings, Unimodal (text), English, 512...",mit,0.067,384,{}
1,sentence-transformers/all-MiniLM-L6-v2,"Text embeddings, Unimodal (text), English, 256...",apache-2.0,0.090,384,{}
2,BAAI/bge-small-zh-v1.5,"Text embeddings, Unimodal (text), Chinese, 512...",mit,0.090,512,{}
3,snowflake/snowflake-arctic-embed-xs,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.090,384,{}
4,jinaai/jina-embeddings-v2-small-en,"Text embeddings, Unimodal (text), English, 819...",apache-2.0,0.120,512,{}


### Q6. Indexing with qdrant

In [38]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [40]:
qd_client = QdrantClient("http://localhost:6333")

In [41]:
model = 'BAAI/bge-small-en'

In [50]:
collection_name = "zoomcamp-hw"

In [51]:
qd_client.delete_collection(collection_name=collection_name)

True

In [52]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=512,
        distance=models.Distance.COSINE
    )
)

True

In [53]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [54]:
points = []

for i, doc in enumerate(documents):
    # question + answer
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [55]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [56]:
question = 'I just discovered the course. Can I join now?'

In [57]:
query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=question,
        model=model_handle 
    ),
    limit=5, # top 5 matches for question
    with_payload=True
)

In [62]:
query_points.points[0]

ScoredPoint(id=14, version=2, score=0.8620738, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)

In [61]:
query_points.points[0].score

0.8620738